In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install rioxarray

import numpy as np
import random
import os
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

data = np.load("/content/drive/MyDrive/new_modis_data.npz")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 102.5 MB/s eta 0:00:00


# Training Data


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation, UpSampling2D,
    Concatenate, Layer, Dropout, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
)
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import EfficientNetB4, MobileNetV2, EfficientNetB0, MobileNetV3Large, VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras.metrics import MeanIoU
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import KFold
import uuid
import time
import pickle

In [3]:
X_train = data["X_train"]
y_train = data["y_train"]

X_val = data["X_val"]
y_val = data["y_val"]

X_test = data["X_test"]
y_test = data["y_test"]

X_train.shape
data.close()

In [ ]:
input_shape = (32, 32, 3)

In [ ]:
X_train.shape

(11080, 32, 32, 3)

In [5]:
recall = tf.keras.metrics.Recall(name='recall')
precision = tf.keras.metrics.Precision(name='precision')

In [4]:
X_train_new = np.concatenate([X_train, X_val], axis=0)
y_train_new = np.concatenate([y_train, y_val], axis=0)

## EfficientNetB0 - Ashley

In [ ]:
def create_efficientnetb0():
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    return model

In [ ]:
efficientnetb0 = create_efficientnetb0()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

efficientnetb0.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

efficientnetb0.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=20, callbacks=callbacks)

Epoch 1/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6575 - loss: 0.6514 - precision: 0.2939 - recall: 0.0094
Epoch 1: val_loss improved from inf to 0.63697, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 43s 33ms/step - accuracy: 0.6575 - loss: 0.6514 - precision: 0.2939 - recall: 0.0094 - val_accuracy: 0.6668 - val_loss: 0.6370 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6651 - loss: 0.6441 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2: val_loss improved from 0.63697 to 0.63651, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.6651 - loss: 0.6441 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6668 - val_loss: 0.6365 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/20
690/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6703 - loss: 0.6383 - precision: 0.0000e

In [ ]:
best_efficientnetb0 = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_efficientnetb0.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.6811 - loss: 0.6266 - precision: 0.0000e+00 - recall: 0.0000e+00


[0.6364513039588928, 0.0, 0.0, 0.6667569279670715]

## MobileNetV2 - Shirina

In [ ]:
def create_mobilenetv2():
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    return model

In [ ]:
mobilenetv2 = create_mobilenetv2()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

mobilenetv2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

mobilenetv2.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=50, callbacks=callbacks)

<ipython-input-10-7b699983fec9>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')


Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6719 - loss: 0.6056 - precision: 0.5175 - recall: 0.1285
Epoch 1: val_loss improved from inf to 0.51378, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - accuracy: 0.6720 - loss: 0.6056 - precision: 0.5178 - recall: 0.1287 - val_accuracy: 0.7545 - val_loss: 0.5138 - val_precision: 0.7051 - val_recall: 0.4525 - learning_rate: 0.0010
Epoch 2/50
687/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7568 - loss: 0.5145 - precision: 0.7108 - recall: 0.4569
Epoch 2: val_loss improved from 0.51378 to 0.49678, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7568 - loss: 0.5145 - precision: 0.7108 - recall: 0.4570 - val_accuracy: 0.7607 - val_loss: 0.4968 - val_precision: 0.7433 - val_recall: 0.4305 - learning_rate: 0.0010
Epoch 3/50
688/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7666 - loss: 0.4949 - precision: 0.7335 - recall: 0.4725
Epoch 3: val_loss

In [ ]:
best_mobilenetv2 = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_mobilenetv2.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8059 - loss: 0.4381 - precision: 0.7686 - recall: 0.5608


[0.4421641528606415, 0.5434606075286865, 0.77073734998703, 0.7939902544021606]

## SqueezeNet - Ashley

In [ ]:
def fire_module(x, squeeze, expand):
    x = layers.Conv2D(squeeze, (1,1), activation='relu', padding='same')(x)
    left = layers.Conv2D(expand, (1,1), activation='relu', padding='same')(x)
    right = layers.Conv2D(expand, (3,3), activation='relu', padding='same')(x)
    return layers.concatenate([left, right], axis=-1)

def create_squeezenet(input_shape=(32, 32, 3), squeeze=16):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(96, (7,7), strides=(2,2), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
    x = fire_module(x, squeeze, 64)
    x = fire_module(x, squeeze, 64)
    x = fire_module(x, squeeze * 2, 128)
    x = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2))(x)
    x = fire_module(x, 32, 128)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=output)

    return model

In [ ]:
squeezenet = create_squeezenet()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

squeezenet.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

squeezenet.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=20, callbacks=callbacks)

Epoch 1/20
692/693 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.7284 - loss: 0.5354 - precision: 0.8479 - recall: 0.6168
Epoch 1: val_loss improved from inf to 0.32407, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 95s 58ms/step - accuracy: 0.7286 - loss: 0.5352 - precision: 0.8478 - recall: 0.6169 - val_accuracy: 0.8714 - val_loss: 0.3241 - val_precision: 0.8987 - val_recall: 0.6921 - learning_rate: 0.0010
Epoch 2/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.8854 - loss: 0.3015 - precision: 0.8619 - recall: 0.7730
Epoch 2: val_loss improved from 0.32407 to 0.27172, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 37s 52ms/step - accuracy: 0.8854 - loss: 0.3015 - precision: 0.8620 - recall: 0.7730 - val_accuracy: 0.8971 - val_loss: 0.2717 - val_precision: 0.8782 - val_recall: 0.8026 - learning_rate: 0.0010
Epoch 3/20
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8956 - loss: 0.2716 - precision: 0.8935 - recall: 0.7834
Epoch 3: val

In [ ]:
best_squeezenet = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_squeezenet.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9428 - loss: 0.1732 - precision: 0.9524 - recall: 0.8629


[0.16746650636196136,
 0.8740860819816589,
 0.9547471404075623,
 0.9442338943481445]

### CV

In [ ]:
batch_sizes = [8, 16, 32]
squeeze_values = [8, 16, 32]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_recall_queezenet = -np.inf
best_config_queezenet = None
best_squeezenet = None

for batch_size in batch_sizes:
    for squeeze in squeeze_values:
          val_acc, val_rec, val_prec = [], [], []
          for train_index, val_index in kf.split(X_train_new):
              X_train, X_val = X_train_new[train_index], X_train_new[val_index]
              y_train, y_val = y_train_new[train_index], y_train_new[val_index]

              squeezenet = create_squeezenet(squeeze=squeeze)

              callbacks = [
                  ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
                  ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=0),
                  EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=0),
              ]

              squeezenet.compile(
                  optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[recall, precision, 'accuracy']
              )

              squeezenet.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=20, callbacks=callbacks, verbose=0)

              best_model = load_model("best_model.keras", custom_objects={
                  'recall': recall,
                  'precision': precision
              })

              scores = best_model.evaluate(X_val, y_val, verbose=0)

              val_acc.append(scores[3])
              val_rec.append(scores[1])
              val_prec.append(scores[2])

          if np.mean(val_rec) > best_recall_queezenet:
              best_recall_queezenet = np.mean(val_rec)
              best_config_queezenet = {
                  'batch_size': batch_size,
                  'squeeze': squeeze,
                  'accuracy': np.mean(val_acc),
                  'recall': np.mean(val_rec),
                  'precision': np.mean(val_prec)
              }
              best_squeezenet = best_model



Epoch 1: val_loss improved from inf to 0.33170, saving model to best_model.keras

Epoch 2: val_loss improved from 0.33170 to 0.29955, saving model to best_model.keras

Epoch 3: val_loss improved from 0.29955 to 0.21581, saving model to best_model.keras

Epoch 4: val_loss improved from 0.21581 to 0.21263, saving model to best_model.keras

Epoch 5: val_loss did not improve from 0.21263

Epoch 6: val_loss improved from 0.21263 to 0.17855, saving model to best_model.keras

Epoch 7: val_loss did not improve from 0.17855

Epoch 8: val_loss did not improve from 0.17855

Epoch 9: val_loss did not improve from 0.17855

Epoch 10: val_loss improved from 0.17855 to 0.16800, saving model to best_model.keras

Epoch 11: val_loss did not improve from 0.16800

Epoch 12: val_loss did not improve from 0.16800

Epoch 13: val_loss did not improve from 0.16800

Epoch 14: val_loss did not improve from 0.16800

Epoch 15: val_loss did not improve from 0.16800

Epoch 1: val_loss improved from inf to 0.33177, s

In [ ]:
print("Best configuration for SuqeezeNet:", best_config_queezenet)

Best configuration for SuqeezeNet: {'batch_size': 32, 'squeeze': 32, 'accuracy': np.float64(0.9514008641242981), 'recall': np.float64(0.8972260594367981), 'precision': np.float64(0.9544788479804993)}


In [ ]:
best_squeezenet.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9410 - loss: 0.1622 - precision: 0.9278 - recall: 0.8830


[0.15424826741218567,
 0.8976441621780396,
 0.9324894547462463,
 0.9442338943481445]

## VGGNet - Ashley

In [ ]:
def build_vgg(input_shape=(32, 32, 3)):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)


In [ ]:
vgg = build_vgg()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

vgg.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[recall, precision, 'accuracy']
)

vgg.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=50, callbacks=callbacks)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8269 - loss: 0.4163 - precision: 0.9017 - recall: 0.7320
Epoch 1: val_loss improved from inf to 0.24397, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.8270 - loss: 0.4162 - precision: 0.9017 - recall: 0.7320 - val_accuracy: 0.9136 - val_loss: 0.2440 - val_precision: 0.9262 - val_recall: 0.8050 - learning_rate: 0.0010
Epoch 2/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9169 - loss: 0.2291 - precision: 0.9118 - recall: 0.8296
Epoch 2: val_loss improved from 0.24397 to 0.21743, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9169 - loss: 0.2291 - precision: 0.9118 - recall: 0.8296 - val_accuracy: 0.9207 - val_loss: 0.2174 - val_precision: 0.9114 - val_recall: 0.8440 - learning_rate: 0.0010
Epoch 3/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9254 - loss: 0.2016 - precision: 0.9276 - recall: 0.8454
Epoch 3: val_los

In [ ]:
best_vgg = load_model("best_model.keras", custom_objects={
    'recall': recall,
    'precision': precision
})

best_vgg.evaluate(X_test, y_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9501 - loss: 0.1571 - precision: 0.9593 - recall: 0.8800


[0.16228215396404266,
 0.8813972473144531,
 0.9576346278190613,
 0.9474824070930481]

### cv-Shirina

In [ ]:
# =================== (1) Configure paths ===================
MODEL_FILE = '/content/drive/MyDrive/best_vgg_model.keras'
CONFIG_FILE = '/content/drive/MyDrive/best_vgg_config.pkl'

# =================== (2) Check files ===================
if os.path.exists(MODEL_FILE) and os.path.exists(CONFIG_FILE):
    print("Detected saved model, loading automatically!")
    best_vgg_model = load_model(
        MODEL_FILE, custom_objects={'recall': recall, 'precision': precision}
    )
    with open(CONFIG_FILE, "rb") as f:
        best_config_vgg = pickle.load(f)
    print("Loaded best parameters:", best_config_vgg)
else:
    print("No saved model detected, retraining...")

    # ======== Hyperparameter settings ========
    batch_sizes = [8, 16, 32]
    learning_rates = [1e-3, 1e-4]
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    best_recall_vgg = -np.inf
    best_config_vgg = None
    best_vgg_model = None

    total_combinations = len(batch_sizes) * len(learning_rates)
    finished_combinations = 0
    total_start_time = time.time()

    for batch_size in batch_sizes:
        for lr in learning_rates:
            group_start_time = time.time()
            val_acc, val_rec, val_prec = [], [], []

            for fold, (train_index, val_index) in enumerate(kf.split(X_train_new)):
                X_train_fold, X_val_fold = X_train_new[train_index], X_train_new[val_index]
                y_train_fold, y_val_fold = y_train_new[train_index], y_train_new[val_index]

                model = build_vgg()
                model.compile(
                    optimizer=Adam(learning_rate=lr),
                    loss='binary_crossentropy',
                    metrics=[recall, precision, 'accuracy']
                )

                callbacks = [
                    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=0),
                    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=0),
                ]

                model.fit(
                    X_train_fold, y_train_fold,
                    validation_data=(X_val_fold, y_val_fold),
                    batch_size=batch_size,
                    epochs=20,
                    callbacks=callbacks,
                    verbose=1
                )

                scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
                val_acc.append(scores[3])
                val_rec.append(scores[1])
                val_prec.append(scores[2])

            mean_rec = np.mean(val_rec)
            if mean_rec > best_recall_vgg:
                best_recall_vgg = mean_rec
                best_config_vgg = {
                    'batch_size': batch_size,
                    'learning_rate': lr,
                    'accuracy': np.mean(val_acc),
                    'recall': mean_rec,
                    'precision': np.mean(val_prec)
                }
                best_vgg_model = model  # Note: the model from the last fold (not saved per fold)

            # ETA progress bar
            finished_combinations += 1
            group_time = time.time() - group_start_time
            elapsed = time.time() - total_start_time
            avg_group_time = elapsed / finished_combinations
            remaining_combinations = total_combinations - finished_combinations
            eta = avg_group_time * remaining_combinations
            print(f"[{finished_combinations}/{total_combinations}] Finished batch_size={batch_size}, lr={lr} in {group_time/60:.2f} min. Estimated remaining: {eta/60:.2f} min ({eta:.1f} sec)")

    # ========== Save model and parameters ==========
    best_vgg_model.save(MODEL_FILE)
    with open(CONFIG_FILE, "wb") as f:
        pickle.dump(best_config_vgg, f)
    print("Best model and parameters saved:", best_config_vgg)


🟡 未检测到模型，将重新训练...
Epoch 1/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.8223 - loss: 0.4049 - precision: 0.8064 - recall: 0.6202 - val_accuracy: 0.9191 - val_loss: 0.2172 - val_precision: 0.9335 - val_recall: 0.8038 - learning_rate: 0.0010
Epoch 2/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9192 - loss: 0.2166 - precision: 0.9200 - recall: 0.8303 - val_accuracy: 0.9249 - val_loss: 0.1969 - val_precision: 0.9523 - val_recall: 0.8049 - learning_rate: 0.0010
Epoch 3/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9281 - loss: 0.1915 - precision: 0.9352 - recall: 0.8402 - val_accuracy: 0.9283 - val_loss: 0.1910 - val_precision: 0.9854 - val_recall: 0.7869 - learning_rate: 0.0010
Epoch 4/20
1478/1478 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9373 - loss: 0.1782 - precision: 0.9534 - recall: 0.8558 - val_accuracy: 0.9398 - val_loss: 0.1638 - val_precision: 0.9659 - val_recall: 0.8409 - learning_rate: 0.0010
Epoch 5/20
1478/1478 ━━━━━━━━━━━━

In [ ]:
#Evaluate on the test set using the best model
scores = best_vgg_model.evaluate(X_test, y_test)
print(f"Test Set Metrics: loss: {scores[0]:.4f} - recall: {scores[1]:.4f} - precision: {scores[2]:.4f} - accuracy: {scores[3]:.4f}")


116/116 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9500 - loss: 0.1594 - precision: 0.9576 - recall: 0.8826
Test Set Metrics: loss: 0.1632 - recall: 0.8757 - precision: 0.9574 - accuracy: 0.9456


## Simple CNN - Rebecca

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Recall
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),

    layers.Dense(128, activation='relu'),

    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### monitor = recall

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_recall',mode='max',patience=5,restore_best_weights=True,verbose=1),
    ModelCheckpoint('best_model_recall.keras',monitor='val_recall',mode='max',save_best_only=True,verbose=1),
    ReduceLROnPlateau(monitor='val_recall',mode='max',factor=0.5,patience=3,min_lr=1e-6,verbose=1)
]
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8057 - loss: 0.4441 - precision: 0.7589 - recall: 0.5762
Epoch 1: val_recall improved from -inf to 0.84728, saving model to best_model_recall.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.8057 - loss: 0.4440 - precision: 0.7590 - recall: 0.5764 - val_accuracy: 0.9017 - val_loss: 0.2488 - val_precision: 0.8563 - val_recall: 0.8473 - learning_rate: 0.0010
Epoch 2/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9123 - loss: 0.2368 - precision: 0.9195 - recall: 0.8095
Epoch 2: val_recall improved from 0.84728 to 0.86190, saving model to best_model_recall.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.9123 - loss: 0.2368 - precision: 0.9196 - recall: 0.8095 - val_accuracy: 0.9231 - val_loss: 0.1984 - val_precision: 0.9030 - val_recall: 0.8619 - learning_rate: 0.0010
Epoch 3/50
691/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9323 - loss: 0.1865 - precision: 0.9520 - recall: 0.84

In [ ]:
model = load_model('best_model_recall.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])
results = model.evaluate(X_test, y_test, batch_size=16, verbose=1)

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9327 - loss: 0.2523 - precision: 0.8620 - recall: 0.9394


### monitor = loss

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss',mode='min',patience=5,restore_best_weights=True,verbose=1),
    ModelCheckpoint('best_model_loss.keras',monitor='val_loss',mode='min',save_best_only=True,verbose=1),
    ReduceLROnPlateau(monitor='val_loss',mode='min',factor=0.5,patience=3,min_lr=1e-6,verbose=1)
]
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9792 - loss: 0.0596 - precision: 0.9767 - recall: 0.9603
Epoch 1: val_loss improved from inf to 0.20770, saving model to best_model_loss.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.9792 - loss: 0.0596 - precision: 0.9767 - recall: 0.9603 - val_accuracy: 0.9448 - val_loss: 0.2077 - val_precision: 0.9355 - val_recall: 0.8960 - learning_rate: 0.0010
Epoch 2/50
687/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9808 - loss: 0.0571 - precision: 0.9826 - recall: 0.9594
Epoch 2: val_loss did not improve from 0.20770
693/693 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9808 - loss: 0.0571 - precision: 0.9826 - recall: 0.9594 - val_accuracy: 0.9461 - val_loss: 0.2092 - val_precision: 0.9503 - val_recall: 0.8846 - learning_rate: 0.0010
Epoch 3/50
692/693 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9826 - loss: 0.0531 - precision: 0.9842 - recall: 0.9626
Epoch 3: val_loss did not improve from 0.20770
693/6

In [ ]:
model = load_model('best_model_loss.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])
results = model.evaluate(X_test, y_test, batch_size=16, verbose=1)

231/231 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9486 - loss: 0.1865 - precision: 0.9294 - recall: 0.9072


### Use CV to find the best design of CNN

In [6]:
def build_model(version='A'):
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))

    if version == 'A':
        convs = [32, 64, 128]
        dense = 128
        dropout = None
    elif version == 'B':
        convs = [32, 64]
        dense = 64
        dropout = None
    elif version == 'C':
        convs = [64, 128, 256]
        dense = 256
        dropout = 0.3
    elif version == 'D':
        convs = [32, 64, 64]
        dense = 128
        dropout = 0.5

    for c in convs:
        model.add(layers.Conv2D(c, (3,3), activation='relu', padding='same'))
        model.add(layers.MaxPooling2D(2,2))

    model.add(layers.Flatten())
    if dropout:
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dense, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', Recall(name='recall'), Precision(name='precision')]
    )
    return model

#### cross validation to find the best design

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

model_versions = ['A', 'B', 'C', 'D']
results = {}

for version in model_versions:
    print(f"\nEvaluating Model Version {version}")
    all_scores = {'accuracy': [], 'recall': [], 'precision': []}

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_new, y_train_new)):
        print(f"  Fold {fold+1}")

        X_train, X_val = X_train_new[train_idx], X_train_new[val_idx]
        y_train, y_val = y_train_new[train_idx], y_train_new[val_idx]

        model = build_model(version)

        callbacks = [
            EarlyStopping(monitor='val_recall', mode='max', patience=5, restore_best_weights=True, verbose=0),
            ReduceLROnPlateau(monitor='val_recall', mode='max', factor=0.5, patience=3, min_lr=1e-6, verbose=0),
        ]

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=16,
            callbacks=callbacks,
            verbose=0
        )

        scores = model.evaluate(X_val, y_val, verbose=0)
        all_scores['accuracy'].append(scores[1])
        all_scores['recall'].append(scores[2])
        all_scores['precision'].append(scores[3])

    results[version] = {
        'accuracy': np.mean(all_scores['accuracy']),
        'recall': np.mean(all_scores['recall']),
        'precision': np.mean(all_scores['precision']),
    }


Evaluating Model Version A
  Fold 1
  Fold 2
  Fold 3
  Fold 4
  Fold 5

Evaluating Model Version B
  Fold 1
  Fold 2
  Fold 3
  Fold 4
  Fold 5

Evaluating Model Version C
  Fold 1
  Fold 2
  Fold 3
  Fold 4
  Fold 5

Evaluating Model Version D
  Fold 1
  Fold 2
  Fold 3
  Fold 4
  Fold 5


#### print results and model summary

In [ ]:
print("\nCV Results：")
for version, metrics in results.items():
    print(f"Model {version}: acc={metrics['accuracy']:.4f}, recall={metrics['recall']:.4f}, precision={metrics['precision']:.4f}")
best_version = max(results.items(), key=lambda x: x[1]['recall'])[0]
print(f"\nVersion of the best design is {best_version}, based on Recall")


CV Results：
Model A: acc=0.9557, recall=0.9357, precision=0.9317
Model B: acc=0.9291, recall=0.9464, precision=0.8642
Model C: acc=0.9368, recall=0.9342, precision=0.8864
Model D: acc=0.9542, recall=0.9268, precision=0.9375

Version of the best design is B, based on Recall


In [10]:
best_model = build_model(best_version)
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       262,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 281,665 (1.07 MB)

 Trainable params: 281,665 (1.07 MB)

 Non-trainable params: 0 (0.00 B)

#### train the best model on the entire train set

In [40]:
best_model.fit(
    X_train_new, y_train_new,
    validation_split=0.1,
    epochs=50,
    batch_size=16,
    callbacks=[
        ModelCheckpoint('best_cnn.keras',monitor='val_recall',mode='max',save_best_only=True,verbose=1),
        EarlyStopping(monitor='val_recall', mode='max', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_recall', mode='max', factor=0.5, patience=3, min_lr=1e-6)
    ],
    verbose=1
)

Epoch 1/50
819/831 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9780 - loss: 0.0677 - precision: 0.9817 - recall: 0.9524
Epoch 1: val_recall improved from -inf to 0.92184, saving model to best_cnn.keras
831/831 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9779 - loss: 0.0678 - precision: 0.9817 - recall: 0.9523 - val_accuracy: 0.9398 - val_loss: 0.1590 - val_precision: 0.9020 - val_recall: 0.9218 - learning_rate: 2.5000e-04
Epoch 2/50
814/831 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9785 - loss: 0.0678 - precision: 0.9821 - recall: 0.9523
Epoch 2: val_recall did not improve from 0.92184
831/831 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9785 - loss: 0.0678 - precision: 0.9821 - recall: 0.9523 - val_accuracy: 0.9486 - val_loss: 0.1486 - val_precision: 0.9273 - val_recall: 0.9198 - learning_rate: 2.5000e-04
Epoch 3/50
819/831 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9806 - loss: 0.0603 - precision: 0.9838 - recall: 0.9569
Epoch 3: val_recall did not improve from 0.921

#### use performance on testset as the final performance evaluation result of simple cnn

In [41]:
model = load_model('best_cnn.keras')
model.save('/content/drive/MyDrive/best_cnn_model.keras')

In [42]:
results = model.evaluate(X_test, y_test, batch_size=16, verbose=1)

231/231 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9565 - loss: 0.1352 - precision: 0.9266 - recall: 0.9374


## InceptionV3 - Vicki

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau
import tensorflow.keras.backend as K


def build_inception_model():
    inputs = Input(shape=(32, 32, 3))
    x = Resizing(128, 128)(inputs)
    base = InceptionV3(weights=None, include_top=False, input_tensor=x)
    x = GlobalAveragePooling2D()(base.output)
    output = Dense(1, activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=output)


In [ ]:
inception = build_inception_model()

callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
]

inception.compile(
    optimizer=Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)

inception.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.7392 - loss: 0.5307 - precision: 0.6299 - recall: 0.5651
Epoch 1: val_loss improved from inf to 0.34998, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 186s 153ms/step - accuracy: 0.7392 - loss: 0.5306 - precision: 0.6301 - recall: 0.5652 - val_accuracy: 0.8609 - val_loss: 0.3500 - val_precision: 0.8030 - val_recall: 0.7717 - learning_rate: 1.0000e-04
Epoch 2/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8624 - loss: 0.3403 - precision: 0.8098 - recall: 0.7478
Epoch 2: val_loss improved from 0.34998 to 0.24251, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 109s 75ms/step - accuracy: 0.8624 - loss: 0.3403 - precision: 0.8099 - recall: 0.7479 - val_accuracy: 0.9082 - val_loss: 0.2425 - val_precision: 0.8919 - val_recall: 0.8245 - learning_rate: 1.0000e-04
Epoch 3/50
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.8963 - loss: 0.2774 - precision: 0.8698 - recall: 0.8104


CV

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

acc_list, recall_list, precision_list = [], [], []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
    print(f"Fold {fold+1}")

    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    inception = build_inception_model()

    callbacks = [
        ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", verbose=1),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
    ]

    inception.compile(
        optimizer=Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
    )

    inception.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=16,
        callbacks=callbacks,
        verbose=1
    )

    best_inception = load_model("best_model.keras",
                            custom_objects={'Precision': Precision, 'Recall': Recall})

    scores = best_inception.evaluate(X_test, y_test, verbose=1)

    #scores = model.evaluate(X_val, y_val, verbose=0)
    acc_list.append(scores[3])        # accuracy
    recall_list.append(scores[1])     # recall
    precision_list.append(scores[2])  # precision

print(f"Average Accuracy: {np.mean(acc_list):.4f}")
print(f"Average Recall: {np.mean(recall_list):.4f}")
print(f"Average Precision: {np.mean(precision_list):.4f}")


Fold 1
Epoch 1/10
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7642 - loss: 0.5169 - precision: 0.6690 - recall: 0.6065
Epoch 1: val_loss improved from inf to 0.41968, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 157s 123ms/step - accuracy: 0.7643 - loss: 0.5168 - precision: 0.6690 - recall: 0.6065 - val_accuracy: 0.8010 - val_loss: 0.4197 - val_precision: 0.7381 - val_recall: 0.6161 - learning_rate: 1.0000e-04
Epoch 2/10
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8527 - loss: 0.3673 - precision: 0.8039 - recall: 0.7367
Epoch 2: val_loss improved from 0.41968 to 0.25376, saving model to best_model.keras
693/693 ━━━━━━━━━━━━━━━━━━━━ 67s 65ms/step - accuracy: 0.8528 - loss: 0.3672 - precision: 0.8040 - recall: 0.7367 - val_accuracy: 0.8998 - val_loss: 0.2538 - val_precision: 0.8674 - val_recall: 0.8224 - learning_rate: 1.0000e-04
Epoch 3/10
693/693 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8954 - loss: 0.2753 - precision: 0.8758 - recall: 0

In [ ]:
best_inception = load_model("best_model.keras",
                        custom_objects={'Precision': Precision, 'Recall': Recall})

results = best_inception.evaluate(X_test, y_test, verbose=1)


116/116 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - accuracy: 0.9487 - loss: 0.1587 - precision: 0.9314 - recall: 0.9066


## California Case - Bimo

In [46]:
data_cal = np.load("/content/drive/MyDrive/cal_fires_128x128.npz")
X_cal, y_cal = data_cal['X_data'], data_cal['y_data']

In [47]:
# Resize to (32, 32, 3) to match model input
X_cal_ready = tf.image.resize(X_cal, [32, 32]).numpy()


In [48]:
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Recall, Precision

# Load model with custom_objects
model = load_model('/content/drive/MyDrive/best_cnn_model.keras',
                   custom_objects={'recall': Recall(), 'precision': Precision()})

# Recompile loaded model in terms for evaluation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', Recall(name='recall'), Precision(name='precision')])

# Evaluate in California Y
results = model.evaluate(X_cal_ready, y_cal, batch_size=16, verbose=1)

# Evaluation Matrices
print(f"\nAccuracy : {results[1]:.4f}")
print(f"Recall   : {results[2]:.4f}")
print(f"Precision: {results[3]:.4f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.7446 - loss: 2.5692 - precision: 0.7085 - recall: 0.8086

Accuracy : 0.7167
Recall   : 0.8393
Precision: 0.6528
